In [ ]:
import pyforest

import catboost as catt
import lightgbm as lgb
import xgboost as xgb


from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold

In [ ]:
train=pd.read_csv('Datasets/Train.csv', parse_dates=['Date'])
test=pd.read_csv('Datsets/Test.csv', parse_dates=['Date'])

display(train.head())

In [ ]:
test.head()

In [ ]:
train.shape, test.shape

In [ ]:
drop_columns=['target_min','target_max','target_variance','target_count']
print("The shape before dropping was:", train.shape)
train=train.drop(columns=drop_columns)
print("The shape after dropping is:", train.shape)

Sorting the data based on the date


In [ ]:
train=train.sort_values(by=['Date']).reset_index(drop=True)
test=test.sort_values(by=['Date']).reset_index(drop=True)